# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../output_data/cities.csv')
weather_df.head(3)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Max Temp K,Wind Speed
0,0,Hualmay,35,PE,1593224740,85,-11.10,-77.61,62,289.62,3.67
1,1,Hithadhoo,99,MV,1593224741,71,-0.60,73.08,84,302.24,2.32
2,2,Pacific Grove,90,US,1593224741,82,36.62,-121.92,61,289.15,4.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create data with nice weather conditions
nice_weather_df = weather_df[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & 
                             (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)] 


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# initialize variables
target_lat = []
target_lng = []
humidity = []
hotel_name = []
locate = []
lat1 = []
lng1 = []
h_city = []
h_country = []

# Get latitude and longitude from google maps

for i in range(0, 10): 
    lat = nice_weather_df.iloc[i,6].astype(str)
    lon = nice_weather_df.iloc[i,7].astype(str)
    humid = nice_weather_df.iloc[i,5]
    #city = nice_weather_df.iloc[i,1]
    #country = nice_weather_df.iloc[i,3]
    
    loc = f"{lat}, {lon}"
    
    params = {
    "location": loc,
    "radius": 5000,
    "type": "hotel",
    "key": g_key
    }
  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    lat1 = results[0]["geometry"]['location']["lat"]
    lng1 = results[0]["geometry"]['location']["lng"]
    #loc1 = f"'{lat1}, {lng1}'"
          
    target_lat.append(results[0]["geometry"]['location']["lat"])
    target_lng.append(results[0]["geometry"]['location']["lng"])
    #locate = results[0]["geometry"]['location']["lat"] )
    humidity.append(nice_weather_df.iloc[i,5])
    hotel_name.append(results[0]["name"])
    h_city.append(nice_weather_df.iloc[i,1]) 
    h_country.append(nice_weather_df.iloc[i,3]) 
    
    
# Create dataframe for google maps    
hotel_df = pd.DataFrame({
                        "lat": target_lat,
                        "lng": target_lng,
                        "humid": humidity,
                        "hotel": hotel_name,
                        "city": h_city,
                        "country": h_country
                            
                        })


In [8]:
# Create paramters for mp
locations = hotel_df[["lat", "lng"]].astype(float)
location = hotel_df[["lat", "lng"]].astype(float)
humidity = hotel_df["humid"].astype(float)
city_cntry = hotel_df["city"] + ", "+ hotel_df["country"]
hotel = "Hotel: "+ hotel_df["hotel"]

# Create map
fig = gmaps.figure()

# Add heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)

# Add marker layer
marker_layer = gmaps.marker_layer(locations,hover_text=hotel,info_box_content=city_cntry)
fig.add_layer(marker_layer)

# Display map
fig




Figure(layout=FigureLayout(height='420px'))